In [1]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from itertools import product

In [3]:
def DataFetcher(symbol, interval='1d'):
    suffixes = ['.NS', '.BO']
    for suffix in suffixes:
        try:
            data = yf.download(symbol + suffix, interval=interval, progress=False)
            data.drop(columns='Adj Close', inplace=True)
            data.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True)
            data['Time'] = data.index
            return data
        except Exception as e:
            print(f"Failed to fetch data for {symbol+suffix}: {e}")
    return pd.DataFrame()

In [4]:
def get_supertrend(high, low, close, lookback, multiplier):

    # ATR

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()

    # H/L AVG AND BASIC UPPER & LOWER BAND

    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()

    # FINAL UPPER BAND
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in upper_band - upper_band]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i,0] = 0
        else:
            if (upper_band[i] < final_bands.iloc[i-1,0]) | (close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = upper_band[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]

    # FINAL LOWER BAND

    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i, 1] = 0
        else:
            if (lower_band[i] > final_bands.iloc[i-1,1]) | (close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = lower_band[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]

    # SUPERTREND

    supertrend = pd.DataFrame(columns = [f'supertrend_{lookback}'])
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]

    for i in range(len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]

    supertrend = supertrend.set_index(upper_band.index)
    supertrend = supertrend.dropna()[1:]

    # ST UPTREND/DOWNTREND

    upt = []
    dt = []
    close = close.iloc[len(close) - len(supertrend):]

    for i in range(len(supertrend)):
        if close[i] > supertrend.iloc[i, 0]:
            upt.append(supertrend.iloc[i, 0])
            dt.append(np.nan)
        elif close[i] < supertrend.iloc[i, 0]:
            upt.append(np.nan)
            dt.append(supertrend.iloc[i, 0])
        else:
            upt.append(np.nan)
            dt.append(np.nan)

    st, upt, dt = pd.Series(supertrend.iloc[:, 0]), pd.Series(upt), pd.Series(dt)
    upt.index, dt.index = supertrend.index, supertrend.index

    return st, upt, dt

In [5]:
def implement_st_strategy(df):
    prices=df['close']
    st=df['st']
    st_signal = []
    signal = 0

    for i in range(len(st)):
        if st[i-1] > prices[i-1] and st[i] < prices[i]:
            if signal != 1:
                signal = 1
                st_signal.append(signal)
            else:
                st_signal.append(0)
        elif st[i-1] < prices[i-1] and st[i] > prices[i]:
            if signal != -1:
                signal = -1
                st_signal.append(signal)
            else:
                st_signal.append(0)
        else:
            st_signal.append(0)

    df['st_signal']=st_signal
    position = []
    for i in range(len(st_signal)):
        if st_signal[i] > 1:
            position.append(-1)
        else:
            position.append(1)

    for i in range(len(df['close'])):
        if st_signal[i] == 1:
            position[i] = 1
        elif st_signal[i] == -1:
            position[i] = -1
        else:
            position[i] = position[i-1]
    df['signal']=position
    signal_mapping = {0: 'HOLD', 1: 'BUY', -1: 'SELL'}
    df['action'] = df['st_signal'].map(signal_mapping)

    return df[['Time','close','signal','action']]

In [6]:
def calculate_super_trend_performance(df):
    if 'signal' not in df.columns:
      raise ValueError("Signal column not created. Check calculate_super_trend function.")

    df['returns'] = np.log(df['close'] / df['close'].shift(1))
    df['strategy'] = df['signal'].shift(1) * df['returns']
    df['creturns'] = df['returns'].cumsum().apply(np.exp)
    df['cstrategy'] = df['strategy'].cumsum().apply(np.exp)
    perf = df['cstrategy'].iloc[-1]
    buy_and_hold = df['creturns'].iloc[-1]
    outperf = perf - buy_and_hold
    return round(perf, 6), round(buy_and_hold, 6), round(outperf, 6)

In [7]:
def calculate_super_trend(df, multiplier, period):
  df['st'], df['s_upt'], df['st_dt'] = get_supertrend(df['high'], df['low'], df['close'], period, multiplier)
  df = df[1:]
  df=implement_st_strategy(df)
  return df

In [8]:
def optimize_super_trend_parameters(df, multipliers, periods):
    best_performance = -np.inf
    best_params = None
    for multiplier, period in product(multipliers, periods):
        temp_df = df.copy()
        temp_df=calculate_super_trend(temp_df, multiplier, period)
        performance = calculate_super_trend_performance(temp_df)[0]
        if performance > best_performance:
            best_performance = performance
            best_params = (multiplier, period)
    return best_params

In [9]:
def calculate_profit_percentage(df):
    initial_principal = 100000  # Start with $100,000
    cash = initial_principal
    shares = 0

    for index, row in df.iterrows():
        if row['action'] == 'BUY' and cash > 0:
            shares_bought = cash // row['close']
            shares += shares_bought
            cash -= shares_bought * row['close']
        elif row['action'] == 'SELL' and shares > 0:
            cash += shares * row['close']
            shares = 0

    # If there are any shares left, sell them at the last recorded price
    if shares > 0:
        cash += shares * df['close'].iloc[-1]
        shares = 0

    final_principal = cash
    profit = final_principal - initial_principal
    profit_percentage = (profit / initial_principal) * 100
    return profit_percentage

In [10]:
stocks_df=pd.read_csv('STOCKS_1.csv')
stocks_df = stocks_df.dropna(subset=['Symbol'])
stocks=stocks_df['Symbol']

In [11]:
from tqdm import tqdm
all_stocks_data = []  # Initialize an empty list to store individual stock data
total_stocks = len(stocks)

multipliers = np.arange(1.5, 3.5, 0.5)
periods = np.arange(7, 22, 1)

for stock in tqdm(stocks, desc="Fetching data", unit="stock"):
    try:
        df = DataFetcher(stock)
        best_params = optimize_super_trend_parameters(df.copy(), multipliers, periods)
        all_stocks_data.append({'symbol': stock, 'multiplier': best_params[0], 'period': best_params[1]})
    except:
        continue

all_stocks_df = pd.DataFrame(all_stocks_data)  # Concatenate all stock dataframes


Fetching data:   0%|          | 4/2164 [33:57<327:08:57, 545.25s/stock]
1 Failed download:
['3MINDIA.NS']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))
Fetching data:   1%|          | 15/2164 [1:32:39<112:05:38, 187.78s/stock]

In [ ]:
csv_file_path = 'Paramns_stocks.csv'
all_stocks_df.to_csv(csv_file_path, index=False)